In [21]:
!pip install wget
!pip install scikit-learn 
!pip install "ibm-watson-machine-learning>=1.0.310"

In [22]:
credentials = {
    "url":"https://eu-de.ml.cloud.ibm.com" ,
    "apikey":"JI_XLUnZsrPd4qB79c1uiJav5qj5VwkGDLyJ1LFzyXgU"
}

In [23]:
import os

In [24]:
try:
    project_id=os.environ["PROJECT_ID"]

except KeyError:
    project_id = input("please enter your project_id:")

In [25]:
project_id

'79229c05-b289-4dec-b3ba-7c3e285f1d54'

In [26]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='HOL0NZYlvcOGT3VJpoIkbXH_zdCVg1bOJqZbSS2dV7Ec',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'project1-donotdelete-pr-irr5vaxzptgafr'
object_key = 'legal_sentiment_dataset.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head(10)


,ID,Phrase,Sentiment
0,1,The plaintiff's claims are dismissed.,-1
1,2,The contract is deemed valid and enforceable.,1
2,3,The appeal is denied due to lack of merit.,-1
3,4,The legal team submitted additional evidence.,0
4,5,The appeal is under consideration.,0
5,6,The settlement agreement is fair and reasonable.,1
6,7,The appeal is under consideration.,0
7,8,The settlement agreement is fair and reasonable.,1
8,9,The court reviewed the documentation.,0
9,10,The contract is deemed valid and enforceable.,1


In [27]:
label_map={
    -1:"negative",
    0:"neural",
    1:"positive"
}

In [28]:
data.value_counts(['Sentiment'])

Sentiment
 1           184
-1           167
 0           149
Name: count, dtype: int64

In [29]:
from sklearn.model_selection import train_test_split

data_train,data_test,y_train,y_test=train_test_split(data['Phrase'],
                                                     data['Sentiment'],
                                                     test_size=0.3,
                                                     random_state=33,
                                                     stratify=data['Sentiment'])
print(type(data_train))
data_train=pd.DataFrame(data_train)
data_test=pd.DataFrame(data_test)

<class 'pandas.core.series.Series'>


In [30]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q', 'CODELLAMA_34B_INSTRUCT_HF', 'GRANITE_20B_MULTILINGUAL']


In [31]:
model_id=ModelTypes.FLAN_T5_XXL

In [32]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters={
    GenParams.DECODING_METHOD:"Greedy",
    GenParams.RANDOM_SEED:33,
    GenParams.REPETITION_PENALTY:1,
    GenParams.MIN_NEW_TOKENS:1,
    GenParams.MAX_NEW_TOKENS:1,
    
}

model.get_details()

In [33]:
from ibm_watson_machine_learning.foundation_models import Model

model=Model(

    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id)

In [34]:
model.get_details()

{'model_id': 'google/flan-t5-xxl',
 'label': 'flan-t5-xxl-11b',
 'provider': 'Google',
 'source': 'Hugging Face',
 'functions': [{'id': 'text_generation'}],
 'short_description': 'flan-t5-xxl is an 11 billion parameter model based on the Flan-T5 family.',
 'long_description': 'flan-t5-xxl (11B) is an 11 billion parameter model based on the Flan-T5 family. It is a pretrained T5 - an encoder-decoder model pre-trained on a mixture of supervised / unsupervised tasks converted into a text-to-text format, and fine-tuned on the Fine-tuned Language Net (FLAN) with instructions for better zero-shot and few-shot performance.',
 'terms_url': 'https://huggingface.co/google/flan-t5-xxl/blob/main/README.md',
 'input_tier': 'class_2',
 'output_tier': 'class_2',
 'number_params': '11b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'

In [35]:
instruction="""Determine the sentiment of the sentense. 
Use either 'positive', 'negative', 'neutral'. Use the provided examples as a template."""

In [36]:
zero_shot_input=[{"input":text} for text in data_test['Phrase']]
for i in range(2):
    print(f"The sentence example {i+1} is:\n\t {zero_shot_input[i]['input']}\n")

The sentence example 1 is:
	 The legal team submitted additional evidence.

The sentence example 2 is:
	 The appeal is denied due to lack of merit.



In [37]:
data_train_and_labels=data_train.copy()
data_train_and_labels['Sentiment']=y_train

In [38]:
few_shot_example=[] 
few_shot_examples=[] 
for phrase, sentiment in data_train_and_labels.groupby('Sentiment').apply(lambda x: x.sample (2)).values: 
    few_shot_example.append(f"\tsentence:\t{phrase}\n\tsentiment: {sentiment}\n") 
few_shot_examples=[''.join(few_shot_example)]

In [39]:
few_shot_inputs_ = [{"input":text} for text in data_test['Phrase'].values]

In [41]:
for i in range(2): 
    print(f"The sentence example {i+1} is: \n {few_shot_inputs_[i]['input']}\n") 

The sentence example 1 is: 
 The legal team submitted additional evidence.

The sentence example 2 is: 
 The appeal is denied due to lack of merit.



In [ ]:
results=[]
for inp in few_shot_inputs_[:2]:
    results.append(model.generate(" ".join([instruction+few_shot_example[0],inp['input']]))["results"][0])

In [ ]:
import json
print(json.dumps(results,indent=2))

In [ ]:
y_true=[label_map[label] for label in y_test.values[:2]]
y_true

In [ ]:
y_pred=[result['generated_text'] for result in results]
y_pred